In [1]:
from google.colab import drive

drive.mount('/content/drive')
%cd "/content/drive/My Drive/Project 4"
#"/content/drive/My Drive/UCLA/ECE219/Project 4"

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1cHi87xejaQVuaSSd0K4mlmrUKL5dX2o_/Project 4


### CardiffNLP Roberta (Twitter-roBERTa-base for Sentiment Analysis)

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 11.9 MB/s 
     |████████████████████████████████| 67 kB 3.3 MB/s 
     |████████████████████████████████| 895 kB 35.9 MB/s 
     |████████████████████████████████| 6.5 MB 38.1 MB/s 
     |████████████████████████████████| 596 kB 43.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
from tqdm import tqdm
import torch
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import time
from transformers import pipeline

tqdm.pandas()


In [4]:
tweet_emt_df = pd.read_csv('data/tweet_emt_df.csv', index_col=0, lineterminator='\n')
tweet_emt_df.head(3)

,hashtag,title,tweet.text,citation_date,tweet.lang,tweet.user.name,original_author.name,tweet.retweet_count,tweet.favorite_count,metrics.citations.total,...,sent_nltk_vader_neu_cleaned_hashtag_title,sent_nltk_vader_pos_cleaned_hashtag_title,sent_nltk_vader_compound_cleaned_hashtag_title,sent_nltk_vader_label_cleaned_hashtag_title,sent_textblob_polarity_cleaned_title,sent_textblob_subjectivity_cleaned_title,sent_textblob_label_cleaned_title,sent_textblob_polarity_cleaned_hashtag_title,sent_textblob_subjectivity_cleaned_hashtag_title,sent_textblob_label_cleaned_hashtag_title
205,#gohawks,“@TheDA53: “@nathanSD8: @TheDA53 broncos?! 😢 #...,“@TheDA53: “@nathanSD8: @TheDA53 broncos?! 😢 #...,1421222681,en,Nathan Santo Domingo,Nathan Santo Domingo,0,0,1,...,1.000,0.000,0.0000,neutral,0.0,0.000,neutral,0.0,0.000,neutral
203,#gohawks,Dr. Jim Kurtz &amp; I before Seahawks vs Panth...,Dr. Jim Kurtz &amp; I before Seahawks vs Panth...,1421222750,en,Dr Gerry Ramogida,Dr Gerry Ramogida,0,0,3,...,0.586,0.414,0.8718,positive,0.5,0.625,positive,0.5,0.625,positive
143,#gopatriots,NFL PLAYOFFS: Brady throws 3 TDs in win over R...,NFL PLAYOFFS: Brady throws 3 TDs in win over R...,1421222838,en,TeH pHiLaDeLpHIa iNq,TeH pHiLaDeLpHIa iNq,0,0,1,...,0.678,0.322,0.5859,positive,0.8,0.400,positive,0.8,0.400,positive


##### __Roberta sentiment__ 

In [5]:
# Preprocess text (username and link placeholders)
# Code reference: https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment
def preprocess(text):
    new_text = []
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary
def get_roberta_model():
    task='sentiment'
    MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

    tokenizer = AutoTokenizer.from_pretrained(MODEL)

    # download label mapping
    labels=[]
    mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
    with urllib.request.urlopen(mapping_link) as f:
        html = f.read().decode('utf-8').split("\n")
        csvreader = csv.reader(html, delimiter='\t')
    labels = [row[1] for row in csvreader if len(row) > 1]

    # PT
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    model.save_pretrained(MODEL)
    #model.to(device)
    return model, labels, tokenizer

def sample_usage(model, tokenizer):
    # sample roberta usage
    text = "Good night 😊"
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        print(f"{i+1}) {l} {np.round(float(s), 4)}")
        
def apply_roberta(text, tokenizer):
    # text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True,max_length=50)
    encoded_input.to(device)
    output = model(**encoded_input)
    scores = output[0][0].cpu().detach().numpy()
    scores = softmax(scores)
    return pd.Series(scores)

In [6]:
model, labels, tokenizer = get_roberta_model()
#sample_usage(model, tokenizer)

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [7]:
device = 0 if torch.cuda.is_available() else -1
model.to(device)
model.device

device(type='cuda', index=0)

In [8]:
tweet_emt_df.reset_index(inplace=True)
feat = 'cleaned_title'
tweet_emt_df[feat] = tweet_emt_df[feat].apply(lambda x: 'empty string' if type(x)!=str else x)

In [10]:
prefix ='sent_cardiffnlp_roberta_'
feat = 'cleaned_title'
post_fix='_'+feat
col_names = [prefix+'neg'+post_fix, prefix+'neut'+post_fix, prefix+'pos'+post_fix]
#tweet_emt_df[col_names] = tweet_emt_df.progress_apply(lambda x: apply_roberta(x[feat], tokenizer), axis=1, result_type='expand')

during_game_only = tweet_emt_df[(tweet_emt_df.game_status=='during_game')]
during_game_only[col_names] = during_game_only.progress_apply(lambda x: apply_roberta(x[feat],tokenizer), axis=1, result_type='expand')
during_game_only.to_csv('data/during_game_only_with_cardiffnnlp_roberta.csv')


100%|██████████| 89804/89804 [24:45<00:00, 60.46it/s]
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [11]:
def get_label_name_roberta(txt):
  abv = txt.split('_')[3]
  if abv=='pos': return 'positive'
  if abv=='neut': return 'neutral'
  else: return 'negative'

label_feat_name = prefix+'label'+post_fix

during_game_only[label_feat_name] = during_game_only[col_names].idxmax(axis=1)
during_game_only[label_feat_name].head(10)
during_game_only[label_feat_name] = during_game_only[label_feat_name].apply(lambda x: get_label_name_roberta(x))
during_game_only.to_csv('data/during_game_only_with_cardiffnnlp_roberta.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# using pipeline -- can't get all the scores, only the chosen label's score..
#device = 0 if torch.cuda.is_available() else -1
# use pipeline to do inference
#cardiff_nlp_bert_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device, verbose=10)
#tweets = tweet_emt_df[feat].head(500).tolist()
#res = cardiff_nlp_bert_pipeline(tweets, verbose=10)
#res

### AllenNLP Roberta

PS: Run both Robeerta models separately, there is a conflict between library installs. (before doing this part restart kernel.)

In [ ]:
!pip install allennlp==2.4.0 
!pip install allennlp-models==2.4.0

     |████████████████████████████████| 625 kB 5.4 MB/s 
     |████████████████████████████████| 2.1 MB 30.1 MB/s 
     |████████████████████████████████| 17.4 MB 569 kB/s 
     |████████████████████████████████| 125 kB 44.6 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 1.2 MB 40.8 MB/s 
     |████████████████████████████████| 804.1 MB 2.9 kB/s 
     |████████████████████████████████| 592 kB 47.4 MB/s 
     |████████████████████████████████| 1.8 MB 36.1 MB/s 
     |████████████████████████████████| 132 kB 46.1 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 8.6 MB 35.3 MB/s 
     |████████████████████████████████| 138 kB 50.6 MB/s 
     |████████████████████████████████| 127 kB 47.2 MB/s 
     |████████████████████████████████| 3.3 MB 35.8 MB/s 
     |████████████████████████████████| 895 kB 41.2 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |██████████████

In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
import pandas as pd
from tqdm import tqdm
import torch

tqdm.pandas()

In [ ]:
device = 0 if torch.cuda.is_available() else -1
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/stanford-sentiment-treebank-roberta.2021-03-11.tar.gz", cuda_device=device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Plugin allennlp_models could not be loaded: No module named 'nltk.translate.meteor_score'
downloading: 100%|##########| 1294401432/1294401432 [01:14<00:00, 17272710.98B/s]


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

In [ ]:
predictor.predict("a very well-made, funny and entertaining picture.")

{'label': '1',
 'logits': [4.5721821784973145, -4.596451759338379],
 'probs': [0.9998956918716431, 0.00010424791253171861],
 'token_ids': [0, 102, 182, 157, 12, 7078, 6, 6269, 8, 11110, 2170, 4, 2],
 'tokens': ['<s>',
  'a',
  'Ġvery',
  'Ġwell',
  '-',
  'made',
  ',',
  'Ġfunny',
  'Ġand',
  'Ġentertaining',
  'Ġpicture',
  '.',
  '</s>']}

In [ ]:
def predict_allennlp_roberta(txt):
  res = predictor.predict(txt)
  pos, neg = res['probs'][0], res['probs'][1]
  lbl = 'positive' if res['label']=='1' else 'negative'
  return pos, neg, lbl

In [ ]:
predict_allennlp_roberta("a very well-made, funny and entertaining picture.")

(0.9998956918716431, 0.00010424791253171861, 'positive')

In [ ]:
tweet_emt_df = pd.read_csv('data/tweet_emt_df.csv', index_col=0, lineterminator='\n')
tweet_emt_df.head(3)

,hashtag,title,tweet.text,citation_date,tweet.lang,tweet.user.name,original_author.name,tweet.retweet_count,tweet.favorite_count,metrics.citations.total,...,sent_nltk_vader_neu_cleaned_hashtag_title,sent_nltk_vader_pos_cleaned_hashtag_title,sent_nltk_vader_compound_cleaned_hashtag_title,sent_nltk_vader_label_cleaned_hashtag_title,sent_textblob_polarity_cleaned_title,sent_textblob_subjectivity_cleaned_title,sent_textblob_label_cleaned_title,sent_textblob_polarity_cleaned_hashtag_title,sent_textblob_subjectivity_cleaned_hashtag_title,sent_textblob_label_cleaned_hashtag_title
205,#gohawks,“@TheDA53: “@nathanSD8: @TheDA53 broncos?! 😢 #...,“@TheDA53: “@nathanSD8: @TheDA53 broncos?! 😢 #...,1421222681,en,Nathan Santo Domingo,Nathan Santo Domingo,0,0,1,...,1.000,0.000,0.0000,neutral,0.0,0.000,neutral,0.0,0.000,neutral
203,#gohawks,Dr. Jim Kurtz &amp; I before Seahawks vs Panth...,Dr. Jim Kurtz &amp; I before Seahawks vs Panth...,1421222750,en,Dr Gerry Ramogida,Dr Gerry Ramogida,0,0,3,...,0.586,0.414,0.8718,positive,0.5,0.625,positive,0.5,0.625,positive
143,#gopatriots,NFL PLAYOFFS: Brady throws 3 TDs in win over R...,NFL PLAYOFFS: Brady throws 3 TDs in win over R...,1421222838,en,TeH pHiLaDeLpHIa iNq,TeH pHiLaDeLpHIa iNq,0,0,1,...,0.678,0.322,0.5859,positive,0.8,0.400,positive,0.8,0.400,positive


In [ ]:
feat = 'cleaned_title'
tweet_emt_df.reset_index(inplace=True)
tweet_emt_df[feat] = tweet_emt_df[feat].apply(lambda x: 'empty string' if type(x)!=str else x)

In [ ]:
prefix ='sent_allennlp_roberta_'
feat = 'cleaned_title'
post_fix='_'+feat
col_names = [prefix+'pos'+post_fix, prefix+'neg'+post_fix, prefix+'label'+post_fix,]

during_game_only = tweet_emt_df[(tweet_emt_df.game_status=='during_game')]
during_game_only[col_names] = during_game_only.progress_apply(lambda x: predict_allennlp_roberta(x[feat]), axis=1, result_type='expand')
during_game_only.to_csv('data/during_game_only_with_allennlp_roberta.csv')

100%|██████████| 89804/89804 [1:10:52<00:00, 21.12it/s]
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
'''
import math
prefix ='sent_allennlp_roberta_'
feat = 'cleaned_title'
post_fix='_'+feat
col_names = [prefix+'pos'+post_fix, prefix+'neg'+post_fix, prefix+'label'+post_fix,]

k = 10000
samps = tweet_emt_df.shape[0]
total_batch = math.ceil(samps/k)
for ind, i in enumerate(range(0, samps, k)):  #range(0, tweet_emt_df.shape[0], k)
  print(f'{ind}/{total_batch}')
  stt =i
  end = ((ind+1)*k) 
  end = samps if end > samps else end
  tweet_emt_df[col_names].loc[stt:end] = tweet_emt_df.loc[stt:end].progress_apply(lambda x: predict_allennlp_roberta(x[feat]), axis=1, result_type='expand')
  tweet_emt_df.to_csv('data/tweet_emt_df_with_allennlp_roberta.csv')
'''

"\nprefix ='sent_allennlp_roberta_'\nfeat = 'cleaned_title'\npost_fix='_'+feat\ncol_names = [prefix+'pos'+post_fix, prefix+'neg'+post_fix, prefix+'label'+post_fix,]\n\nk = 10000\nsamps = tweet_emt_df.shape[0]\ntotal_batch = math.ceil(samps/k)\nfor ind, i in enumerate(range(0, samps, k)):  #range(0, tweet_emt_df.shape[0], k)\n  print(f'{ind}/{total_batch}')\n  stt =i\n  end = ((ind+1)*k) \n  end = samps if end > samps else end\n  tweet_emt_df[col_names].loc[stt:end] = tweet_emt_df.loc[stt:end].progress_apply(lambda x: predict_allennlp_roberta(x[feat]), axis=1, result_type='expand')\n  tweet_emt_df.to_csv('data/tweet_emt_df_with_allennlp_roberta.csv')\n"